In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as  sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [108]:
df = pd.read_csv('ds_salaries.csv')

In [109]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [110]:
#Finding the unique values of all the variables
print('Unique Values:')
columns = list(df.columns)
for col in columns:
    u = df[col].nunique()
    print(f'{col} = {u}')

Unique Values:
work_year = 4
experience_level = 4
employment_type = 4
job_title = 93
salary = 815
salary_currency = 20
salary_in_usd = 1035
employee_residence = 78
remote_ratio = 3
company_location = 72
company_size = 3


Based on the unique values, we can conclude the following:
- There are 4 types of employement along with work_year and experience_level.
- There are 93 different types of job profiles in data science.
- Company size can be L - "Large", M - "Medium" and S - "small."
- There are 3 types of working culture, which can be remote, hybrid, in-office.


In [111]:
#finding different unique values in columns
column = df[['experience_level', 'employment_type', 'remote_ratio']]
for col in column:
    Unique = df[col].unique()
    print(f'{col} = {Unique}')

experience_level = ['SE' 'MI' 'EN' 'EX']
employment_type = ['FT' 'CT' 'FL' 'PT']
remote_ratio = [100   0  50]


In [112]:
# replacing abbreviations with relevant and more clear name
df['experience_level'] = df['experience_level'].replace(['EN', 'MI', 'SE', 'EX'],['Entry-Level', 'Mid-Level', 'Senior-Level', 'Executive-Level'])
df['employment_type'] = df['employment_type'].replace(['PT', 'FT', 'FL', 'CT'], 
                                                      ['Part-Time', 'Full-Time', 'Freelance', 'Contract'])
df['remote_ratio'] = df['remote_ratio'].replace([100, 0, 50], ['remote', 'on-site', 'hybrid'])

df['company_size'] = df['company_size'].replace(["L", "M", "S"], ['Large', 'Medium', 'Small'])

In [113]:
df.shape

(3755, 11)

In [114]:
df["company_size"].unique()

array(['Large', 'Small', 'Medium'], dtype=object)

In [115]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
work_year,3755.0,2022.373635,0.691448,2020.0,2022.0,2022.0,2023.0,2023.0
salary,3755.0,190695.571771,671676.500508,6000.0,100000.0,138000.0,180000.0,30400000.0
salary_in_usd,3755.0,137570.389880,63055.625278,5132.0,95000.0,135000.0,175000.0,450000.0


- On the basis of descriptive statistics of the dataframe, we can conclude that we have data points of 4 years starting from the year 2020 to 2023.
- Minimum salary in the field of Data science is 6000 with maximum being as high as 30400000.

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           3755 non-null   int64 
 1   experience_level    3755 non-null   object
 2   employment_type     3755 non-null   object
 3   job_title           3755 non-null   object
 4   salary              3755 non-null   int64 
 5   salary_currency     3755 non-null   object
 6   salary_in_usd       3755 non-null   int64 
 7   employee_residence  3755 non-null   object
 8   remote_ratio        3755 non-null   object
 9   company_location    3755 non-null   object
 10  company_size        3755 non-null   object
dtypes: int64(3), object(8)
memory usage: 322.8+ KB


In [117]:
df.duplicated().sum()

1171

In [118]:
df = df.drop_duplicates()
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,Senior-Level,Full-Time,Principal Data Scientist,80000,EUR,85847,ES,remote,ES,Large
1,2023,Mid-Level,Contract,ML Engineer,30000,USD,30000,US,remote,US,Small
2,2023,Mid-Level,Contract,ML Engineer,25500,USD,25500,US,remote,US,Small
3,2023,Senior-Level,Full-Time,Data Scientist,175000,USD,175000,CA,remote,CA,Medium
4,2023,Senior-Level,Full-Time,Data Scientist,120000,USD,120000,CA,remote,CA,Medium
...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,Senior-Level,Full-Time,Data Scientist,412000,USD,412000,US,remote,US,Large
3751,2021,Mid-Level,Full-Time,Principal Data Scientist,151000,USD,151000,US,remote,US,Large
3752,2020,Entry-Level,Full-Time,Data Scientist,105000,USD,105000,US,remote,US,Small
3753,2020,Entry-Level,Contract,Business Data Analyst,100000,USD,100000,US,remote,US,Large


In [119]:
df.duplicated().sum()

0

## *DATA PREPROCESSING*

In [120]:

# Define a custom function to assign the salary range
def assign_salary_range(salary):
    if 0 <= salary <= 15000:
        return '0-15000'
    elif 15000 < salary <= 30000:
        return '15000-30000'
    elif 30000 < salary <= 45000:
        return '30000-45000'
    elif 45000 < salary <= 450000:
        range_start = int(salary / 15000) * 15000
        range_end = range_start + 15000
        return f'{range_start}-{range_end}'
    else:
        return 'Other'
    
# Create the new column based on the salary range
df['salary_range'] = df['salary_in_usd'].apply(assign_salary_range)




In [121]:
# Assuming your dataset is stored in a DataFrame called 'df'
x = pd.DataFrame(df)  # Replace 'your_data' with your actual data

# Dropping the columns
columns_to_drop = ['work_year', 'salary', 'salary_in_usd', 'employee_residence', 'salary_currency']
x = df.drop(columns=columns_to_drop, axis = 1)


In [122]:
x

,experience_level,employment_type,job_title,remote_ratio,company_location,company_size,salary_range
0,Senior-Level,Full-Time,Principal Data Scientist,remote,ES,Large,75000-90000
1,Mid-Level,Contract,ML Engineer,remote,US,Small,15000-30000
2,Mid-Level,Contract,ML Engineer,remote,US,Small,15000-30000
3,Senior-Level,Full-Time,Data Scientist,remote,CA,Medium,165000-180000
4,Senior-Level,Full-Time,Data Scientist,remote,CA,Medium,120000-135000
...,...,...,...,...,...,...,...
3750,Senior-Level,Full-Time,Data Scientist,remote,US,Large,405000-420000
3751,Mid-Level,Full-Time,Principal Data Scientist,remote,US,Large,150000-165000
3752,Entry-Level,Full-Time,Data Scientist,remote,US,Small,105000-120000
3753,Entry-Level,Contract,Business Data Analyst,remote,US,Large,90000-105000


In [123]:
y = df["salary_in_usd"]

In [124]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd



# Columns to encode
columns_to_encode = ['experience_level', 'employment_type', 'job_title', 'remote_ratio', 'company_location', 'company_size','salary_range']

# Encoding labels
label_encoder = LabelEncoder()
for column in columns_to_encode:
    x[column] = label_encoder.fit_transform(x[column])


print(x)


      experience_level  employment_type  job_title  remote_ratio  \
0                    3                2         84             2   
1                    2                0         66             2   
2                    2                0         66             2   
3                    3                2         47             2   
4                    3                2         47             2   
...                ...              ...        ...           ...   
3750                 3                2         47             2   
3751                 2                2         84             2   
3752                 0                2         47             2   
3753                 0                0         17             2   
3754                 3                2         45             0   

      company_location  company_size  salary_range  
0                   25             0            27  
1                   70             2             4  
2                   70  

In [125]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Linear Regression
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)

# Random Forest Regression
random_forest = RandomForestRegressor()
random_forest.fit(X_train, y_train)

# Predicting on the test set
linear_regression_preds = linear_regression.predict(X_test)
random_forest_preds = random_forest.predict(X_test)

# Evaluating the models
linear_regression_rmse = mean_squared_error(y_test, linear_regression_preds, squared=False)
random_forest_rmse = mean_squared_error(y_test, random_forest_preds, squared=False)

print("Linear Regression RMSE:", linear_regression_rmse)
print("Random Forest RMSE:", random_forest_rmse)

Linear Regression RMSE: 56043.28418001053
Random Forest RMSE: 9203.693772680894


In [126]:
print(df["salary_in_usd"].min())
print(df["salary_in_usd"].max())
print(df["salary_in_usd"].mean())

5132
450000
133409.28018575851


array(['75000-90000', '15000-30000', '165000-180000', '120000-135000',
       '210000-225000', '135000-150000', '90000-105000', '150000-165000',
       '105000-120000', '270000-285000', '225000-240000', '195000-210000',
       '60000-75000', '240000-255000', '330000-345000', '180000-195000',
       '255000-270000', '300000-315000', '45000-60000', '30000-45000',
       '285000-300000', '315000-330000', '0-15000', '345000-360000',
       '420000-435000', '375000-390000', '360000-375000', '405000-420000',
       '450000-465000'], dtype=object)